In [ ]:
#|default_exp audio

# Audio buffer classes and helpers

In [ ]:
#| export
import numpy as np

In [ ]:
#| export
from asr_teach.utils import SAMPLE_RATE, BUFFER_SECONDS, DTYPE

In [ ]:
#| export

class AudioBuffer:
    '''Buffers a numpy array of audio streaming from a socket.
    '''
    def __init__(self,
                 size=SAMPLE_RATE * BUFFER_SECONDS, 
                 dtype=DTYPE):
        
        # initialize the buffer
        self.size = size
        self.data = []#np.zeros(size, dtype=dtype)
        
        # head for data buffer
        self.ptr = 0 
        # holds leftover samples
        self.rem = None 
        # flag for whether the buffer is full
        self.is_full = False 

        
    def accumulate(self, data: np.ndarray):
        '''Reads in and buffers the incoming `data`.
        '''
        # number of incoming samples
        num_samples = len(data)
        
        if len(self.data) >= self.size:
            self.is_full = True
            self.rem = self.data[self.size:]
            self.data = self.data[:self.size]
        else:
            self.data.extend(data)
        
#         # handle buffer overflow
#         if self.ptr + num_samples >= self.size:

#             # flag the buffer as full
#             self.is_full = True

#             # store the leftover samples
#             valid = self.size - self.ptr
#             self.data[self.ptr:] = data[:valid]
#             self.rem = data[valid:]
        
#         # buffer in the data
#         else:
#             self.data[self.ptr: self.ptr + num_samples] = data
#             self.ptr += num_samples

            
    def get_data(self):
        '''Gets the data buffer.
        '''
        return self.data
    
    
    def reset(self):
        '''Resets buffer head for new, incoming data.
        '''
        self.ptr = 0
        self.is_full = False
        self.data = []

        
    def flush(self):
        '''Puts remainder data on the buffer and moves the head forward.
        '''
        if self.rem:
            rem_sz = len(self.rem)
            self.data[:rem_sz] = self.rem
            self.ptr += rem_sz
            self.rem = None